# Project Title: Mood-Task Planner

# Project Outline and Objectives:
<span style="font-size:18px;"> This project's primary goal is to create an easy work planner that enables users to arrange their daily activities according to their energy level or mood, which can be classified as High, Medium, or Low. When deciding which activities to complete first, this planner takes into thought an individual's level of motivation or energy, in contrast to traditional task managers that just evaluate deadlines. Each user will be able to effectively manage their own list of tasks due to the system's user login and customized task storage.</span>

# Methodology and initial Findings
<span style="font-size:18px;">Python 3 and SQLite as the local database are used in the development of this project. The application runs in a console-style manner within a Jupyter Notebook and makes use of modular functionalities and a menu-driven interface. A basic salted hash algorithm is used to safely store passwords. The logic is implemented using loops, conditional statements, and basic Python functions. The system allows tasks to be added, modified, viewed, and deleted based on mood categories; all data is permanently stored in the database.</span>

<span style="font-size:18px;">During the initial phase project, I was searching for a database to store all the project-related records. I discovered SQLite, which is an ideal option for organizing and storing data without the need for external setup. Another finding is that the system becomes easier and more useful by maintaining basic categories based on user mood, such as high, medium, and low. Additionally, the project's basic framework allows future additions of features like task reminder notifications or user productivity tracking.</span>

# Author's Name: Maksuda E Elahi | Date: 09.09.2025

## Import necessary python libraries

In [1]:

# Import necessary modules
# create and manage the database
import sqlite3 
# hashing passwords securely
import hashlib 
# generate random salt values for passwords
import secrets      
# to record timestamps for created tasks
from datetime import datetime  
import getpass
# Define the database file name
db_name = "task_plan.db"

# Function to create and return a connection to the database
def get_connection():
    return sqlite3.connect(db_name)


## Database table creation

In [2]:
def initialize_db():
    # Connect to the database
    conn = get_connection()
    cursor = conn.cursor()

    # Create 'users' table to store account information
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS users (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            username TEXT UNIQUE,
            pass_hash TEXT,
            salt TEXT
        )
    ''')

    # Create 'tasks' table to store task information
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS tasks (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            user_id INTEGER,
            task TEXT,
            task_description TEXT,
            mood_level TEXT,
            status TEXT DEFAULT 'Pending',
            created_at TEXT,
            FOREIGN KEY(user_id) REFERENCES users(id)
        )
    ''')

    # Save changes and close the connection
    conn.commit()
    conn.close()
    print("Database initialized successfully!")

# call the database function
initialize_db()


Database initialized successfully!


## User Account Creation

In [3]:
def user_account(username, password):
    # created a random salt value for password hashing
    salt = secrets.token_hex(8)
    
    # combined  password and salt, then hash it using SHA-256
    pass_hash = hashlib.sha256((password + salt).encode()).hexdigest()
    
    # Connect to database and insert new user data
    conn = get_connection()
    cursor = conn.cursor()
    try:
        cursor.execute("INSERT INTO users (username, pass_hash, salt) VALUES (?, ?, ?)",
                       (username, pass_hash, salt))
        conn.commit()
        print("Account has created successfully!")
    except sqlite3.IntegrityError:
        # Error occurs if the username already exists
        print("Username already exists. Try another usename.")
    conn.close()

## User Login

In [4]:
def user_login(username, password):
    conn = get_connection()
    cursor = conn.cursor()
    
    # Retrieve user info based on given username
    cursor.execute("SELECT id, username, pass_hash, salt FROM users WHERE username = ?", (username,))
    user = cursor.fetchone()
    
    if user:
        user_id, user_name, stored_hash, salt = user
        # Hash the entered password using the stored salt
        input_hash = hashlib.sha256((password + salt).encode()).hexdigest()
        
        if input_hash == stored_hash:
            print(f"✅ Successfully logged in! Welcome, {user_name}.")
            conn.close()
            return user_id
        else:
            print("❌ Incorrect password.")
    else:
        print("⚠️ Username not found.")
    
    conn.close()
    return None


## Add Task Feature

In [5]:
def add_task(user_id, task, task_description, mood_level):
    conn = get_connection()
    cursor = conn.cursor()
    created_at = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    cursor.execute('''INSERT INTO tasks (user_id, task, task_description, mood_level, status, created_at)
                      VALUES (?, ?, ?, ?, 'Pending', ?)''',
                   (user_id, task, task_description, mood_level, created_at))
    conn.commit()
    conn.close()
    print("Task added successfully!")

## View Tasks By Mood

In [6]:
def view_task(user_id, mood_level=None):
    conn = get_connection()
    cursor = conn.cursor()
    
    if mood_level:
        cursor.execute("SELECT id, task, task_description, status FROM tasks WHERE user_id=? AND mood_level=?", 
                       (user_id, mood_level))
    else:
        cursor.execute("SELECT id, task, task_description, status, mood_level FROM tasks WHERE user_id=?", 
                       (user_id,))
    
    rows = cursor.fetchall()
    conn.close()
    
    if not rows:
        print("No tasks found.")
    else:
        print("\n--- Your Tasks ---")
        for row in rows:
            print(f"ID: {row[0]}, Task: {row[1]}, Desc: {row[2]}, Status: {row[3]}")


## Select Task as Completed

In [7]:
def task_completed(task_id, user_id):
    conn = get_connection()
    cursor = conn.cursor()
    cursor.execute("UPDATE tasks SET status='Completed' WHERE id=? AND user_id=?", (task_id, user_id))
    conn.commit()
    conn.close()
    print("Task completed!")


## Edit Task

In [8]:
def edit_task(task_id, user_id, new_task, new_description, new_mood):
    conn = get_connection()
    cursor = conn.cursor()
    cursor.execute('''UPDATE tasks SET task=?, task_description=?, mood_level=? 
                      WHERE id=? AND user_id=?''', 
                   (new_task, new_description, new_mood, task_id, user_id))
    conn.commit()
    conn.close()
    print("Task updated successfully!")


## Delete Task

In [9]:
def delete_task(task_id, user_id):
    conn = get_connection()
    cursor = conn.cursor()
    cursor.execute("DELETE FROM tasks WHERE id=? AND user_id=?", (task_id, user_id))
    conn.commit()
    conn.close()
    print("Task deleted successfully!")


## Display Task Report

In [10]:
def task_report(user_id):
    conn = get_connection()
    cursor = conn.cursor()

    # --- 1️⃣ Get total and completed counts ---
    cursor.execute("SELECT COUNT(*) FROM tasks WHERE user_id=?", (user_id,))
    total = cursor.fetchone()[0]

    cursor.execute("SELECT COUNT(*) FROM tasks WHERE user_id=? AND status='Completed'", (user_id,))
    completed = cursor.fetchone()[0]

    pending = total - completed

    # --- 2️⃣ Fetch task details ---
    cursor.execute("""
        SELECT id, task, task_description, mood_level, status
        FROM tasks
        WHERE user_id=?
        ORDER BY created_at DESC
    """, (user_id,))
    tasks = cursor.fetchall()

    conn.close()

    # --- 3️⃣ Display Summary ---
    print("=" * 60)
    print(f"📋 TASK REPORT for USER ID: {user_id}")
    print("=" * 60)
    print(f" Total Tasks: {total} | ✅ Completed: {completed} | ⏳ Pending: {pending}")
    print("-" * 60)

    # --- 4️⃣ Display Each Task ---
    if tasks:
        for tid, title, desc, mood, status in tasks:
            print(f"🆔 Task ID: {tid}")
            print(f"📌 Title: {title}")
            print(f"💬 Description: {desc}")
            print(f"🎭 Mood Level: {mood}")
            print(f"📅 Status: {status}")
            print("-" * 60)
    else:
        print("No tasks found for this user.")


## Console

In [11]:
def main_menu(user_id):
    while True:
        print("\n--- Mood Task Planner ---")
        print("1. Add Task")
        print("2. View Tasks by Mood")
        print("3. Mark Task as Completed")
        print("4. Edit Task")
        print("5. Delete Task")
        print("6. View Summary")
        print("7. Logout")

        choice = input("Enter your choice: ")

        if choice == '1':
            task = input("Enter task title: ")
            desc = input("Enter description: ")
            mood = input("Enter mood level (High/Medium/Low): ").capitalize()
            add_task(user_id, task, desc, mood)
        elif choice == '2':
            mood = input("Enter mood level (High/Medium/Low or All): ").capitalize()
            view_task(user_id, None if mood == 'All' else mood)
        elif choice == '3':
            tid = int(input("Enter task ID to mark completed: "))
            task_completed(tid, user_id)
        elif choice == '4':
            tid = int(input("Enter task ID to edit: "))
            new_task = input("New title: ")
            new_desc = input("New description: ")
            new_mood = input("New mood level: ").capitalize()
            edit_task(tid, user_id, new_task, new_desc, new_mood)
        elif choice == '5':
            tid = int(input("Enter task ID to delete: "))
            delete_task(tid, user_id)
        elif choice == '6':
            task_report(user_id)
        elif choice == '7':
            print("Logged out successfully!")
            break
        else:
            print("Invalid choice. Try again.")


In [19]:
# ---- Console-based entry point for the Mood Task Planner ----
import getpass
def start_console_app():
    initialize_db()
    print("🧠 Welcome to the Mood Task Planner!")
    
    while True:
        print("\nPlease choose an option:")
        print("1️⃣  Create a New Account")
        print("2️⃣  Login to Existing Account")
        print("3️⃣  Exit")
        
        choice = input("Enter your choice (1/2/3): ").strip()
        
        if choice == "1":
            # Register new user
            username = input("Enter a new username: ")
            password = getpass.getpass("Enter a new password: ")
            user_account(username, password)
            
        elif choice == "2":
            # Login existing user
            username = input("Enter your username: ")
            password = getpass.getpass("Enter your password: ")
            
            user_id = user_login(username, password)
            if user_id:
                print("🎯 Login verified! Opening your planner...")
                main_menu(user_id)   # Launch main planner menu after login
            else:
                print("⚠️ Login failed. Try again.")
                
        elif choice == "3":
            print("👋 Exiting the Mood Task Planner. Goodbye!")
            break
        
        else:
            print("❌ Invalid choice. Please try 1, 2, or 3.")



In [20]:
start_console_app()

Database initialized successfully!
🧠 Welcome to the Mood Task Planner!

Please choose an option:
1️⃣  Create a New Account
2️⃣  Login to Existing Account
3️⃣  Exit


Enter your choice (1/2/3):  2
Enter your username:  Maksuda
Enter your password:  ········


✅ Successfully logged in! Welcome, Maksuda.
🎯 Login verified! Opening your planner...

--- Mood Task Planner ---
1. Add Task
2. View Tasks by Mood
3. Mark Task as Completed
4. Edit Task
5. Delete Task
6. View Summary
7. Logout


Enter your choice:  1
Enter task title:  Jack
Enter description:  2
Enter mood level (High/Medium/Low):  High


Task added successfully!

--- Mood Task Planner ---
1. Add Task
2. View Tasks by Mood
3. Mark Task as Completed
4. Edit Task
5. Delete Task
6. View Summary
7. Logout


...

Logged out successfully!

Please choose an option:
1️⃣  Create a New Account
2️⃣  Login to Existing Account
3️⃣  Exit


Enter your choice (1/2/3):  1
Enter a new username:  Jack
Enter a new password:  ········


Account has created successfully!

Please choose an option:
1️⃣  Create a New Account
2️⃣  Login to Existing Account
3️⃣  Exit


Enter your choice (1/2/3):  3


👋 Exiting the Mood Task Planner. Goodbye!
